<a href="https://colab.research.google.com/github/Raphael-Oliveira-01/Projetos-de-An-lises-financeiras-do-Brasil/blob/main/automa%C3%A7%C3%A3o_testes_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Criando as pastas do projeto
!mkdir -p PROJETO_2_AUTOMACAO_TESTES
!mkdir -p PROJETO_2_AUTOMACAO_TESTES/motor_credito
!mkdir -p PROJETO_2_AUTOMACAO_TESTES/testes/casos_de_teste
!mkdir -p PROJETO_2_AUTOMACAO_TESTES/config

print("Estrutura de pastas criada com sucesso!")

Estrutura de pastas criada com sucesso!


In [2]:
#  Criando os Arquivos do Projeto
# 2.1 - Arquivo de Requisitos
%%writefile PROJETO_2_AUTOMACAO_TESTES/requirements.txt
pandas
pytest
openpyxl

Writing PROJETO_2_AUTOMACAO_TESTES/requirements.txt


In [3]:
# 2.2 - Política de Crédito
%%writefile PROJETO_2_AUTOMACAO_TESTES/config/politica_credito.json
{
    "SCORE_MINIMO_NEGACAO": 300,
    "IDADE_MINIMA": 18,
    "RENDA_MINIMA_NEGACAO": 1500,

    "SCORE_MINIMO_APROVACAO": 750,
    "RENDA_MINIMA_APROVACAO": 5000,
    "IDADE_MINIMA_APROVACAO": 25,

    "SCORE_APROVACAO_DIRETA": 950
}

Writing PROJETO_2_AUTOMACAO_TESTES/config/politica_credito.json


In [4]:
# 2.3 - Motor de Crédito
%%writefile PROJETO_2_AUTOMACAO_TESTES/motor_credito/engine.py
import pandas as pd
import json
import os

def carregar_politica_de_credito(caminho_arquivo: str) -> dict:
    """
    Carrega os parâmetros da política de crédito de um arquivo JSON.
    """
    try:
        with open(caminho_arquivo, 'r') as f:
            politica = json.load(f)
        return politica
    except FileNotFoundError:
        print(f"Erro: Arquivo de política não encontrado em '{caminho_arquivo}'")
        return None
    except json.JSONDecodeError:
        print(f"Erro: O arquivo em '{caminho_arquivo}' não é um JSON válido.")
        return None

def executar_motor_de_analise(df_solicitantes: pd.DataFrame, politica: dict) -> pd.DataFrame:
    """
    Aplica a lógica de decisão a cada solicitante com base na política de crédito.
    """
    if not politica:
        raise ValueError("A política de crédito não foi carregada corretamente.")

    decisoes = []
    justificativas = []

    for _, solicitante in df_solicitantes.iterrows():
        idade = solicitante['idade']
        renda = solicitante['renda_mensal']
        score = solicitante['score_credito']

        if idade < politica['IDADE_MINIMA']:
            decisoes.append('Negado')
            justificativas.append(f"Idade ({idade}) abaixo da mínima permitida ({politica['IDADE_MINIMA']})")

        elif score < politica['SCORE_MINIMO_NEGACAO']:
            decisoes.append('Negado')
            justificativas.append(f"Score de crédito ({score}) abaixo do mínimo para análise ({politica['SCORE_MINIMO_NEGACAO']})")

        elif renda < politica['RENDA_MINIMA_NEGACAO']:
            decisoes.append('Negado')
            justificativas.append(f"Renda mensal (R${renda:.2f}) abaixo da mínima para análise (R${politica['RENDA_MINIMA_NEGACAO']:.2f})")

        elif score >= politica['SCORE_APROVACAO_DIRETA']:
            decisoes.append('Aprovado')
            justificativas.append(f"Score de crédito ({score}) acima do ponto de aprovação direta ({politica['SCORE_APROVACAO_DIRETA']})")

        elif (score >= politica['SCORE_MINIMO_APROVACAO'] and
              renda >= politica['RENDA_MINIMA_APROVACAO'] and
              idade >= politica['IDADE_MINIMA_APROVACAO']):
            decisoes.append('Aprovado')
            justificativas.append("Cliente atende aos critérios combinados de score, renda e idade para aprovação.")

        else:
            decisoes.append('Análise Manual')
            justificativas.append('Não se enquadrou em nenhuma regra automática. Requer revisão humana.')

    df_solicitantes['decisao'] = decisoes
    df_solicitantes['justificativa_regra'] = justificativas

    return df_solicitantes

Writing PROJETO_2_AUTOMACAO_TESTES/motor_credito/engine.py


In [5]:
# 2.4 - Casos de Teste (CSV)

# Aprovação:

%%writefile PROJETO_2_AUTOMACAO_TESTES/testes/casos_de_teste/casos_aprovacao.csv
id_caso,idade,renda_mensal,score_credito,resultado_esperado
APV001,30,8000,800,Aprovado
APV002,45,15000,960,Aprovado
APV003,25,5000,750,Aprovado

Writing PROJETO_2_AUTOMACAO_TESTES/testes/casos_de_teste/casos_aprovacao.csv


In [6]:
# Negação:

%%writefile PROJETO_2_AUTOMACAO_TESTES/testes/casos_de_teste/casos_negacao.csv
id_caso,idade,renda_mensal,score_credito,resultado_esperado
NEG001,17,5000,800,Negado
NEG002,25,3000,250,Negado
NEG003,30,1499.99,700,Negado

Writing PROJETO_2_AUTOMACAO_TESTES/testes/casos_de_teste/casos_negacao.csv


In [7]:
# Casos de Borda:

%%writefile PROJETO_2_AUTOMACAO_TESTES/testes/casos_de_teste/casos_borda.csv
id_caso,idade,renda_mensal,score_credito,resultado_esperado
BORDA001,18,1500,300,Análise Manual
BORDA002,24,10000,800,Análise Manual
BORDA003,25,4999.99,750,Análise Manual
BORDA004,30,6000,749,Análise Manual
BORDA005,18,2000,950,Aprovado

Writing PROJETO_2_AUTOMACAO_TESTES/testes/casos_de_teste/casos_borda.csv


In [8]:
# 2.5 - O Script de Testes (test_regras_credito.py)

%%writefile PROJETO_2_AUTOMACAO_TESTES/testes/test_regras_credito.py
import pytest
import pandas as pd
import sys
import os

# Adiciona o diretório raiz do projeto ao path para encontrar o 'motor_credito'
sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

from motor_credito.engine import executar_motor_de_analise, carregar_politica_de_credito

# Caminhos absolutos para os arquivos
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
POLITICA_PATH = os.path.join(BASE_DIR, 'config/politica_credito.json')
CASOS_TESTE_DIR = os.path.join(BASE_DIR, 'testes/casos_de_teste')

POLITICA_CREDITO = carregar_politica_de_credito(POLITICA_PATH)

def ler_casos_de_teste(caminho_csv: str) -> list:
    df = pd.read_csv(caminho_csv)
    return [tuple(x) for x in df.to_numpy()]

casos_aprovacao = ler_casos_de_teste(os.path.join(CASOS_TESTE_DIR, 'casos_aprovacao.csv'))
casos_negacao = ler_casos_de_teste(os.path.join(CASOS_TESTE_DIR, 'casos_negacao.csv'))
casos_borda = ler_casos_de_teste(os.path.join(CASOS_TESTE_DIR, 'casos_borda.csv'))

@pytest.mark.parametrize("id_caso, idade, renda_mensal, score_credito, resultado_esperado", casos_aprovacao)
def test_regras_de_aprovacao(id_caso, idade, renda_mensal, score_credito, resultado_esperado):
    dados_solicitante = {'idade': [idade], 'renda_mensal': [renda_mensal], 'score_credito': [score_credito]}
    df_teste = pd.DataFrame(dados_solicitante)
    df_resultado = executar_motor_de_analise(df_teste, POLITICA_CREDITO)
    decisao_obtida = df_resultado['decisao'].iloc[0]
    assert decisao_obtida == resultado_esperado

@pytest.mark.parametrize("id_caso, idade, renda_mensal, score_credito, resultado_esperado", casos_negacao)
def test_regras_de_negacao(id_caso, idade, renda_mensal, score_credito, resultado_esperado):
    dados_solicitante = {'idade': [idade], 'renda_mensal': [renda_mensal], 'score_credito': [score_credito]}
    df_teste = pd.DataFrame(dados_solicitante)
    df_resultado = executar_motor_de_analise(df_teste, POLITICA_CREDITO)
    decisao_obtida = df_resultado['decisao'].iloc[0]
    assert decisao_obtida == resultado_esperado

@pytest.mark.parametrize("id_caso, idade, renda_mensal, score_credito, resultado_esperado", casos_borda)
def test_regras_de_borda(id_caso, idade, renda_mensal, score_credito, resultado_esperado):
    dados_solicitante = {'idade': [idade], 'renda_mensal': [renda_mensal], 'score_credito': [score_credito]}
    df_teste = pd.DataFrame(dados_solicitante)
    df_resultado = executar_motor_de_analise(df_teste, POLITICA_CREDITO)
    decisao_obtida = df_resultado['decisao'].iloc[0]
    assert decisao_obtida == resultado_esperado

Writing PROJETO_2_AUTOMACAO_TESTES/testes/test_regras_credito.py


In [12]:
# 2.6 - O Executor Principal (main_executor_testes.py)

%%writefile PROJETO_2_AUTOMACAO_TESTES/main_executor_testes.py
import pytest
from datetime import datetime
import os

def executar_testes_e_gerar_relatorio():
    print("="*58)
    print("==  AUTOMAÇÃO DE TESTES PARA VALIDAÇÃO DE REGRAS DE CRÉDITO ==")
    print("="*58)
    print(f"\nIniciando a execução dos testes em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    caminho_testes = "testes/"
    args = [caminho_testes, "-v"]

    # Navega para o diretório do projeto antes de rodar os testes
    os.chdir('PROJETO_2_AUTOMACAO_TESTES')

    exit_code = pytest.main(args)

    print("\n--- Análise do Relatório de Testes ---")

    if exit_code == 0:
        print("\n SUCESSO! Todos os casos de teste passaram.")
        print("A lógica do motor de crédito está funcionando conforme os cenários esperados.")
    elif exit_code == 1:
        print("\n FALHA! Foram encontradas inconsistências nos casos de teste.")
        print("Uma ou mais regras não se comportaram como o esperado.")
    else:
        print(f"\n ATENÇÃO! A execução terminou com um código de erro: {exit_code}.")

    print("\n" + "="*58)
    print("==                  Execução Finalizada                 ==")
    print("="*58)

if __name__ == "__main__":
    executar_testes_e_gerar_relatorio()

Overwriting PROJETO_2_AUTOMACAO_TESTES/main_executor_testes.py


In [13]:
# Passo 3: Instalando as Dependências

!pip install -r PROJETO_2_AUTOMACAO_TESTES/requirements.txt

In [14]:
# Passo 4: Executando a Automação de Testes

!python PROJETO_2_AUTOMACAO_TESTES/main_executor_testes.py

==  AUTOMAÇÃO DE TESTES PARA VALIDAÇÃO DE REGRAS DE CRÉDITO ==

Iniciando a execução dos testes em: 2025-08-19 19:55:18

============================= test session starts ==============================
platform linux -- Python 3.12.11, pytest-8.4.1, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content/PROJETO_2_AUTOMACAO_TESTES
plugins: anyio-4.10.0, langsmith-0.4.14, typeguard-4.4.4
collected 11 items                                                             

testes/test_regras_credito.py::test_regras_de_aprovacao[APV001-30-8000-800-Aprovado] PASSED [  9%]
testes/test_regras_credito.py::test_regras_de_aprovacao[APV002-45-15000-960-Aprovado] PASSED [ 18%]
testes/test_regras_credito.py::test_regras_de_aprovacao[APV003-25-5000-750-Aprovado] PASSED [ 27%]
testes/test_regras_credito.py::test_regras_de_negacao[NEG001-17-5000.0-800-Negado] PASSED [ 36%]
testes/test_regras_credito.py::test_regras_de_negacao[NEG002-25-3000.0-250-Negado] PASSED [ 45%]
testes/test_regras